In [0]:
#@title Imports
import tensorflow as tf
from PIL import Image
import numpy as np
import IPython
from matplotlib import image
from matplotlib import pyplot as plt
import pickle
import scipy.io
import imageio
import datetime, os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#@markdown Parameters
ngf = 64 # Number of filters in first layer of generator 
ndf = 64 # Number of filters in first layer of discriminator 
stddev = 0.02 # std of Gaussian distribution
batch_size = 1 # batch_size 
pool_size = 50 # pool_size 
img_width = 64 # Imput image will of width 256 
img_height = 64 # Input image will be of height 256 
img_depth = 3 # RGB format
to_restore = False
vgg_layers = scipy.io.loadmat('/content/gdrive/My Drive/MSc ML/0091/vgg19.mat')["layers"]
check_dir = '/content/gdrive/My Drive/MSc ML/0091/64x64/checkpoint/'

In [0]:
!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard

     |████████████████████████████████| 89.7MB 34.9MB/s 
     |████████████████████████████████| 3.8MB 32.1MB/s 
     |████████████████████████████████| 460kB 40.5MB/s 
ERROR: tensorflow 2.0.0a0 has requirement tb-nightly<1.14.0a20190302,>=1.14.0a20190301, but you'll have tb-nightly 1.15.0a20190903 which is incompatible.


In [0]:
def _weights(layer_idx):
    W = vgg_layers[0][layer_idx][0][0][2][0][0]
    b = vgg_layers[0][layer_idx][0][0][2][0][1]
    return W, b.reshape(b.size)
  
  
def conv2d_relu(prev_layer, layer_idx, layer_name):
    with tf.variable_scope(layer_name):
        # load parameters
        W, b = _weights(layer_idx)
        # initialize parameters
        W = tf.constant(W, name="weights")
        b = tf.constant(b, name="bias")
        # convolution 
        conv2d = tf.nn.conv2d(input=prev_layer,
                              filter=W,
                              strides=[1, 1, 1, 1],
                              padding="SAME")
        # activation
        out = tf.nn.relu(conv2d + b)
    return out
    
    
def avgpool(prev_layer, layer_name):
    with tf.variable_scope(layer_name):
        # average pooling
        out = tf.nn.avg_pool(value=prev_layer,
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1],
                             padding="SAME")
    return out
  
  
def _gram_matrix(F, N, M):
    F = tf.reshape(F, (M, N))
    return tf.matmul(tf.transpose(F), F)
  
  
def _single_style_loss(a, g):
    N = a.shape[3]
    M = a.shape[1] * a.shape[2]
    A = _gram_matrix(a, N, M)
    G = _gram_matrix(g, N, M)
    return tf.reduce_sum(tf.squared_difference(G,A))/((2 * tf.cast(N,tf.float32) * tf.cast(M,tf.float32)) ** 2)
  
  
def _content_loss(P, F):
    return tf.reduce_mean(tf.square(F - P)) / 4.0
  
  
def general_conv2d(inputconv, num_features=64, 
                   window_height=7, window_width=7, 
                   stride_height=1, stride_width=1, 
                   padding_size=1, name="conv2d"):
    with tf.variable_scope(name):
        paddings = tf.constant([[0, 0], [padding_size, padding_size,], [padding_size, padding_size], [0, 0]])
        inputconv_ = tf.pad(inputconv, paddings, "REFLECT")
        conv = tf.contrib.layers.conv2d(inputconv_, num_features, [window_height, window_width], 
                                        [stride_height, stride_width], padding='VALID', activation_fn=None, 
                                        weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                                        biases_initializer=tf.constant_initializer(0.0))
        return conv

In [0]:
def feature_extractor(input_image, name="extractor"):
    with tf.variable_scope(name):
        conv1_1 = conv2d_relu(input_image, 0, "conv1_1")
        conv1_2 = conv2d_relu(conv1_1, 2, "conv1_2")
        avgpool1 = avgpool(conv1_2, "avgpool1")
        conv2_1 = conv2d_relu(avgpool1, 5, "conv2_1")
        conv2_2 = conv2d_relu(conv2_1, 7, "conv2_2")
        avgpool2 = avgpool(conv2_2, "avgpool2")
        conv3_1 = conv2d_relu(avgpool2, 10, "conv3_1")
        conv3_2 = conv2d_relu(conv3_1, 12, "conv3_2")
        conv3_3 = conv2d_relu(conv3_2, 14, "conv3_3")
        conv3_4 = conv2d_relu(conv3_3, 16, "conv3_4")
        avgpool3 = avgpool(conv3_4, "avgpool3")
        conv4_1 = conv2d_relu(avgpool3, 19, "conv4_1")
        conv4_2 = conv2d_relu(conv4_1, 21, "conv4_2")
        conv4_3 = conv2d_relu(conv4_2, 23, "conv4_3")
        conv4_4 = conv2d_relu(conv4_3, 25, "conv4_4")
        avgpool4 = avgpool(conv4_4, "avgpool4")
        conv5_1 = conv2d_relu(avgpool4, 28, "conv5_1")
        return conv1_1, conv2_1, conv3_1, conv4_1, conv3_2, conv5_1

In [0]:
def feature_translator(input_image, name="translator"):
    with tf.variable_scope(name):
        conv1_1 = conv2d_relu(input_image, 0, "conv1_1")
        conv1_2 = conv2d_relu(conv1_1, 2, "conv1_2")
        avgpool1 = avgpool(conv1_2, "avgpool1")
        conv2_1 = conv2d_relu(avgpool1, 5, "conv2_1")
        conv2_2 = conv2d_relu(conv2_1, 7, "conv2_2")
        avgpool2 = avgpool(conv2_2, "avgpool2")
        conv3_1 = conv2d_relu(avgpool2, 10, "conv3_1")
        conv3_2 = conv2d_relu(conv3_1, 12, "conv3_2")
        conv3_3 = conv2d_relu(conv3_2, 14, "conv3_3")
        conv3_4 = conv2d_relu(conv3_3, 16, "conv3_4")
        avgpool3 = avgpool(conv3_4, "avgpool3")
        conv4_1 = conv2d_relu(avgpool3, 19, "conv4_1")
        conv4_2 = conv2d_relu(conv4_1, 21, "conv4_2")
        conv4_3 = conv2d_relu(conv4_2, 23, "conv4_3")
        conv4_4 = conv2d_relu(conv4_3, 25, "conv4_4")
        avgpool4 = avgpool(conv4_4, "avgpool4")
        conv5_1 = conv2d_relu(avgpool4, 28, "conv5_1")
        
        conv1_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv1_1, units=64, name="conv1_1_1")))
        conv1_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv1_1_1, units=64, name="conv1_1_2")))
        conv1_1_ = tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv1_1_2, units=64, name="conv1_1_3"))
        
        conv2_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv2_1, units=128, name="conv2_1_1")))
        conv2_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv2_1_1, units=128, name="conv2_1_2")))
        conv2_1_ = tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv2_1_2, units=128, name="conv2_1_3"))
        
        conv3_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv3_1, units=256, name="conv3_1_1")))
        conv3_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv3_1_1, units=256, name="conv3_1_2")))
        conv3_1_ = tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv3_1_2, units=256, name="conv3_1_3"))
        
        conv4_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv4_1, units=512, name="conv4_1_1")))
        conv4_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv4_1_1, units=512, name="conv4_1_2")))
        conv4_1_ = tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv4_1_2, units=512, name="conv4_1_3"))
        
        conv4_2_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv3_2, units=256, name="conv4_2_1")))
        conv4_2_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv4_2_1, units=256, name="conv4_2_2")))
        conv4_2_ = tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv4_2_2, units=256, name="conv4_2_3"))
        
        conv5_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv5_1, units=512, name="conv5_1_1")))
        conv5_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv5_1_1, units=512, name="conv5_1_2")))
        conv5_1_ = tf.contrib.layers.instance_norm(tf.layers.dense(inputs=conv5_1_2, units=512, name="conv5_1_3"))
        
        return conv1_1_, conv2_1_, conv3_1_, conv4_1_, conv4_2_, conv5_1_     

In [0]:
def feature_translator(input_image, name="translator"):
    with tf.variable_scope(name):
        conv1_1 = conv2d_relu(input_image, 0, "conv1_1")
        conv1_2 = conv2d_relu(conv1_1, 2, "conv1_2")
        avgpool1 = avgpool(conv1_2, "avgpool1")
        conv2_1 = conv2d_relu(avgpool1, 5, "conv2_1")
        conv2_2 = conv2d_relu(conv2_1, 7, "conv2_2")
        avgpool2 = avgpool(conv2_2, "avgpool2")
        conv3_1 = conv2d_relu(avgpool2, 10, "conv3_1")
        conv3_2 = conv2d_relu(conv3_1, 12, "conv3_2")
        conv3_3 = conv2d_relu(conv3_2, 14, "conv3_3")
        conv3_4 = conv2d_relu(conv3_3, 16, "conv3_4")
        avgpool3 = avgpool(conv3_4, "avgpool3")
        conv4_1 = conv2d_relu(avgpool3, 19, "conv4_1")
        conv4_2 = conv2d_relu(conv4_1, 21, "conv4_2")
        conv4_3 = conv2d_relu(conv4_2, 23, "conv4_3")
        conv4_4 = conv2d_relu(conv4_3, 25, "conv4_4")
        avgpool4 = avgpool(conv4_4, "avgpool4")
        conv5_1 = conv2d_relu(avgpool4, 28, "conv5_1")
        
        conv1_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv1_1, 64, 3, 3, 1, 1, 1, name="conv1_1_1")))
        conv1_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv1_1_1, 64, 3, 3, 1, 1, 1, name="conv1_1_2")))
        conv1_1_ = tf.contrib.layers.instance_norm(general_conv2d(conv1_1_2, 64, 3, 3, 1, 1, 1, name="conv1_1_"))
        
        conv2_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv2_1, 128, 3, 3, 1, 1, 1, name="conv2_1_1")))
        conv2_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv2_1_1, 128, 3, 3, 1, 1, 1, name="conv2_1_2")))
        conv2_1_ = tf.contrib.layers.instance_norm(general_conv2d(conv2_1_2, 128, 3, 3, 1, 1, 1, name="conv2_1_"))
        
        conv3_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv3_1, 256, 3, 3, 1, 1, 1, name="conv3_1_1")))
        conv3_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv3_1_1, 256, 3, 3, 1, 1, 1, name="conv3_1_2")))
        conv3_1_ = tf.contrib.layers.instance_norm(general_conv2d(conv3_1_2, 256, 3, 3, 1, 1, 1, name="conv3_1_"))
        
        conv4_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv4_1, 512, 3, 3, 1, 1, 1, name="conv4_1_1")))
        conv4_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv4_1_1, 512, 3, 3, 1, 1, 1, name="conv4_1_2")))
        conv4_1_ = tf.contrib.layers.instance_norm(general_conv2d(conv4_1_2, 512, 3, 3, 1, 1, 1, name="conv4_1_"))
        
        conv3_2_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv3_2, 256, 3, 3, 1, 1, 1, name="conv3_2_1")))
        conv3_2_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv3_2_1, 256, 3, 3, 1, 1, 1, name="conv3_2_2")))
        conv3_2_ = tf.contrib.layers.instance_norm(general_conv2d(conv3_2_2, 256, 3, 3, 1, 1, 1, name="conv3_2_"))
        
        conv5_1_1 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv5_1, 512, 3, 3, 1, 1, 1, name="conv5_1_1")))
        conv5_1_2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(conv5_1_1, 512, 3, 3, 1, 1, 1, name="conv5_1_2")))
        conv5_1_ = tf.contrib.layers.instance_norm(general_conv2d(conv5_1_2, 512, 3, 3, 1, 1, 1, name="conv5_1_"))
        
        return conv1_1_, conv2_1_, conv3_1_, conv4_1_, conv3_2_, conv5_1_     

In [0]:
tf.reset_default_graph()

In [0]:
input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_A")
input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_B")

conv1_1_B, conv2_1_B, conv3_1_B, conv4_1_B, conv4_2_B, conv5_1_B = feature_translator(input_A, name="translatorAtoB")
conv1_1B, conv2_1B, conv3_1B, conv4_1B, conv4_2B, conv5_1B = feature_extractor(input_B, name="extractor")

conv1_1_A, conv2_1_A, conv3_1_A, conv4_1_A, conv4_2_A, conv5_1_A = feature_translator(input_B, name="translatorBtoA")
conv1_1A, conv2_1A, conv3_1A, conv4_1A, conv4_2A, conv5_1A = feature_extractor(input_A, name="extractor")

In [0]:
# loss
with tf.name_scope('loss_AtoB'):
    loss_AtoB_1 = _single_style_loss(conv1_1_B,conv1_1B)
    loss_AtoB_2 = _single_style_loss(conv2_1_B,conv2_1B)
    loss_AtoB_3 = _single_style_loss(conv3_1_B,conv3_1B)
    loss_AtoB_4 = _single_style_loss(conv4_1_B,conv4_1B)
    loss_AtoB_5 = _single_style_loss(conv5_1_B,conv5_1B)
    loss_AtoB_c = _content_loss(conv4_2_B,conv4_2B)
    loss_AtoB = tf.reduce_mean(0.01*(0.5*loss_AtoB_1+loss_AtoB_2+1.5*loss_AtoB_3+3.0*loss_AtoB_4+4.0*loss_AtoB_5)+1000*loss_AtoB_c)
    tf.summary.scalar('loss_AtoB', loss_AtoB)

with tf.name_scope('loss_BtoA'):
    loss_BtoA_1 = _single_style_loss(conv1_1_A,conv1_1A)
    loss_BtoA_2 = _single_style_loss(conv2_1_A,conv2_1A)
    loss_BtoA_3 = _single_style_loss(conv3_1_A,conv3_1A)
    loss_BtoA_4 = _single_style_loss(conv4_1_A,conv4_1A)
    loss_BtoA_5 = _single_style_loss(conv5_1_A,conv5_1A)
    loss_BtoA_c = _content_loss(conv4_2_A,conv4_2A)
    loss_BtoA = tf.reduce_mean(0.01*(0.5*loss_BtoA_1+loss_BtoA_2+1.5*loss_BtoA_3+3.0*loss_BtoA_4+4.0*loss_BtoA_5)+1000*loss_BtoA_c)
    tf.summary.scalar('loss_BtoA', loss_BtoA)

In [0]:
# optimizer
model_vars = tf.trainable_variables()
AtoB_vars = [var for var in model_vars if 'translatorAtoB' in var.name]
BtoA_vars = [var for var in model_vars if 'translatorBtoA' in var.name]

lr = tf.placeholder(tf.float32, shape=[], name="lr")
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
AtoB_trainer = optimizer.minimize(loss_AtoB, var_list=AtoB_vars)
BtoA_trainer = optimizer.minimize(loss_BtoA, var_list=BtoA_vars)

In [0]:
with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Portrait2', 'rb') as f:
    A_input = pickle.load(f)
with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Photo2', 'rb') as f:
    B_input = pickle.load(f)    
    
A_input = np.asarray(A_input)
A_input = A_input.astype('float32')
B_input = np.asarray(B_input)
B_input = B_input.astype('float32')

In [0]:
with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Painting', 'rb') as f:
    C_input = pickle.load(f)    
    
C_input = np.asarray(C_input)
C_input = C_input.astype('float32')

In [0]:
merged = tf.summary.merge_all()
saver = tf.train.Saver()
loss_A2B_list = []
loss_B2A_list = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('/content/gdrive/My Drive/MSc ML/0091/64x64/logs', sess.graph)
    
    if to_restore:
        # chkpt_fname = tf.train.latest_checkpoint(check_dir)
        saver.restore(sess, '/content/gdrive/My Drive/MSc ML/0091/64x64/checkpoint/featuretrans-1400')
      
    for epoch in range(1001):
        print('epoch' + str(epoch))
        # Define the learning rate schedule. The learning rate is kept constant upto 100 epochs and then linearly decayed to 0.
        if(epoch < 900) :
            # learning rate = 2
            curr_lr = 2
        else:
            curr_lr = 2 - 2*(epoch-900)/100
            
        loss_A2B = 0 
        loss_B2A = 0
    
        # Running the training loop for all batches
        for ptr in range(0, 350):
            
            # Train generator G_A->B
            _, __, loss_AtoB_temp, loss_BtoA_temp, summary = sess.run([AtoB_trainer, BtoA_trainer, loss_AtoB, loss_BtoA, merged], 
                                                             feed_dict={input_A:A_input[ptr], input_B:B_input[ptr], 
                                                                        lr:curr_lr})

            loss_A2B += loss_AtoB_temp/350
            loss_B2A += loss_BtoA_temp/350
            
            writer.add_summary(summary, epoch * 350 + ptr)

        print(' loss AtoB' + str(loss_A2B))
        print(' loss BtoA' + str(loss_B2A))
        tf.summary.scalar('loss AtoB', loss_A2B)
        tf.summary.scalar('loss BtoA', loss_B2A)
        loss_A2B_list.append(loss_A2B)
        loss_B2A_list.append(loss_B2A)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/A2B', 'wb') as f:
            pickle.dump(loss_A2B_list, f)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/B2A', 'wb') as f:
            pickle.dump(loss_B2A_list, f)
            
                
        # save training images
        if epoch%200==0:
            saver.save(sess, check_dir + "featuretrans", global_step=epoch) # 1000-0.01-3

    writer.close()

epoch0
 loss AtoB99978144.79428561
 loss BtoA45865250.505714275
epoch1
 loss AtoB59291216.73142858
 loss BtoA29057281.19428572
epoch2
 loss AtoB56894098.08
 loss BtoA27892029.779999997
epoch3
 loss AtoB54108148.25714287
 loss BtoA26315630.494285718
epoch4
 loss AtoB51861737.211428575
 loss BtoA25247248.005714312
epoch5
 loss AtoB50097605.782857165
 loss BtoA24404903.822857134
epoch6
 loss AtoB48284037.977142826
 loss BtoA23646403.082857143
epoch7
 loss AtoB46780819.48000001
 loss BtoA22892653.207142863
epoch8
 loss AtoB45286825.19428571
 loss BtoA21982886.288571417
epoch9
 loss AtoB43965382.451428555
 loss BtoA21146204.78142856
epoch10
 loss AtoB42747437.84000005
 loss BtoA20411335.59857143
epoch11
 loss AtoB41523386.7885714
 loss BtoA19767511.42857143
epoch12
 loss AtoB40439232.88571429
 loss BtoA19186254.950000003
epoch13
 loss AtoB39462723.291428566
 loss BtoA18644283.217142846
epoch14
 loss AtoB38603719.268571414
 loss BtoA18160946.222857136
epoch15
 loss AtoB37792050.265714295
 lo

KeyboardInterrupt: ignored

In [0]:
with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/A2B', 'rb') as f:
    loss_B2A = pickle.load(f)

In [0]:
class VGG(object):
    def __init__(self, input_img):
        # load vgg19
        self.vgg_layers = scipy.io.loadmat('/content/gdrive/My Drive/MSc ML/0091/vgg19.mat')["layers"]
        self.input_img = input_img
        # mean of vgg19
        self.mean_pixels = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3))

    def _weights(self, layer_idx, expected_layer_name):
        """
        get pre-trained weights of each layer
        
        :param layer_idx: layer id in VGG
        :param expected_layer_name: layer name
        :return: pre-trained weight W and b
        """
        W = self.vgg_layers[0][layer_idx][0][0][2][0][0]
        b = self.vgg_layers[0][layer_idx][0][0][2][0][1]
        # name of current layer
        layer_name = self.vgg_layers[0][layer_idx][0][0][0][0]
        assert layer_name == expected_layer_name, print("Layer name error!")

        return W, b.reshape(b.size)

    def conv2d_relu(self, prev_layer, layer_idx, layer_name):
        """
        :param prev_layer: previous layer
        :param layer_idx: layer id in VGG
        :param layer_name: layer name
        """
        with tf.variable_scope(layer_name):
            # load parameters
            W, b = self._weights(layer_idx, layer_name)
            # initialize parameters
            W = tf.constant(W, name="weights")
            b = tf.constant(b, name="bias")
            # convolution 
            conv2d = tf.nn.conv2d(input=prev_layer,
                                  filter=W,
                                  strides=[1, 1, 1, 1],
                                  padding="SAME")
            # activation
            out = tf.nn.relu(conv2d + b)
        setattr(self, layer_name, out)

    def avgpool(self, prev_layer, layer_name):
        """
        average pooling层（这里参考了原论文中提到了avg-pooling比max-pooling效果好，所以采用avg-pooling）
        
        :param prev_layer: 前一层网络（卷积层）
        :param layer_name: 当前layer命名
        """
        with tf.variable_scope(layer_name):
            # average pooling
            out = tf.nn.avg_pool(value=prev_layer,
                                 ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1],
                                 padding="SAME")

        setattr(self, layer_name, out)

    def load(self):
        """
        加载pre-trained的数据
        """
        self.conv2d_relu(self.input_img, 0, "conv1_1")
        self.conv2d_relu(self.conv1_1, 2, "conv1_2")
        self.avgpool(self.conv1_2, "avgpool1")
        self.conv2d_relu(self.avgpool1, 5, "conv2_1")
        self.conv2d_relu(self.conv2_1, 7, "conv2_2")
        self.avgpool(self.conv2_2, "avgpool2")
        self.conv2d_relu(self.avgpool2, 10, "conv3_1")
        self.conv2d_relu(self.conv3_1, 12, "conv3_2")
        self.conv2d_relu(self.conv3_2, 14, "conv3_3")
        self.conv2d_relu(self.conv3_3, 16, "conv3_4")
        self.avgpool(self.conv3_4, "avgpool3")
        self.conv2d_relu(self.avgpool3, 19, "conv4_1")
        self.conv2d_relu(self.conv4_1, 21, "conv4_2")
        self.conv2d_relu(self.conv4_2, 23, "conv4_3")
        self.conv2d_relu(self.conv4_3, 25, "conv4_4")
        self.avgpool(self.conv4_4, "avgpool4")
        self.conv2d_relu(self.avgpool4, 28, "conv5_1")
        self.conv2d_relu(self.conv5_1, 30, "conv5_2")
        self.conv2d_relu(self.conv5_2, 32, "conv5_3")
        self.conv2d_relu(self.conv5_3, 34, "conv5_4")
        self.avgpool(self.conv5_4, "avgpool5")

In [0]:
class LatentTransfer(object):
    def __init__(self, initial_img, conv1_1_, conv2_1_, conv3_1_, conv4_1_, conv5_1_, conv4_2_, img_width, img_height, ptr):

        self.conv1_1_ = conv1_1_
        self.conv2_1_ = conv2_1_
        self.conv3_1_ = conv3_1_
        self.conv4_1_ = conv4_1_
        self.conv4_2_ = conv4_2_
        self.conv5_1_ = conv5_1_
        self.img_width = img_width
        self.img_height = img_height
        self.ptr = ptr
        self.initial_img = initial_img
        
        # feature layer
        self.content_layer = "conv3_2"
        self.style_layers = ["conv1_1", "conv2_1", "conv3_1", "conv4_1", "conv5_1"]

        # weights of content loss and style loss
        self.content_w = 1000
        self.style_w = 0.01

        # weights of style layers
        self.style_layer_w = [0.5, 1.0, 1.5, 3.0, 4.0]

        # global step and learning rate
        self.gstep = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")  # global step
        self.lr = 2.0


    def create_input(self):
        with tf.variable_scope("input"):
            self.input_img = tf.get_variable("in_img", 
                                             shape=([1, self.img_height, self.img_width, 3]),
                                             dtype=tf.float32,
                                             initializer=tf.zeros_initializer())

    def load_vgg(self):
        self.vgg = VGG(self.input_img)
        self.vgg.load()

    def _content_loss(self, P, F):
        """
        :param P: feature map of content image
        :param F: feature map of generated image
        """
        self.content_loss = tf.reduce_sum(tf.square(F - P)) / (4.0 * tf.cast(P.shape[0],tf.float32) * tf.cast(P.shape[1],tf.float32) * tf.cast(P.shape[2],tf.float32))
        
    def _gram_matrix(self, F, N, M):
        """
        :param F: feature map
        :param N: third dimension of feature map
        :param M: first x second dimension of feature map
        :return: Gram Matrix of F
        """
        F = tf.reshape(F, (M, N))

        return tf.matmul(tf.transpose(F), F)

    def _single_style_loss(self, a, g):
        """
        :param a: feature map of style image
        :param g: feature map of generated image
        :return: style loss
        """
        N = a.shape[3]
        M = a.shape[1] * a.shape[2]

        # Gram Matrix of feature maps
        A = self._gram_matrix(a, N, M)
        G = self._gram_matrix(g, N, M)

        return tf.reduce_sum(tf.square(G - A)) / ((2 * tf.cast(N,tf.float32) * tf.cast(M,tf.float32)) ** 2)

    def _style_loss(self):
        # style layer: conv1_1, conv2_1, conv3_1, conv4_1, conv5_1
        E_1 = self._single_style_loss(self.conv1_1_, getattr(self.vgg, self.style_layers[0]))
        E_2 = self._single_style_loss(self.conv2_1_, getattr(self.vgg, self.style_layers[1]))
        E_3 = self._single_style_loss(self.conv3_1_, getattr(self.vgg, self.style_layers[2]))
        E_4 = self._single_style_loss(self.conv4_1_, getattr(self.vgg, self.style_layers[3]))
        E_5 = self._single_style_loss(self.conv5_1_, getattr(self.vgg, self.style_layers[4]))
        # weighted sum
        self.style_loss = 0.5*E_1+1.0*E_2+1.5*E_3+3.0*E_4+4.0*E_5

    def losses(self):
        """
        total loss
        """
        with tf.variable_scope("losses"):
            # contents loss
            with tf.Session() as sess:
                gen_img_content = getattr(self.vgg, self.content_layer)
            self._content_loss(self.conv4_2_, gen_img_content)

            # style loss 
            self._style_loss()
            
            # total loss
            self.total_loss = self.content_w * self.content_loss + self.style_w * self.style_loss

    def optimize(self):
        self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.total_loss, global_step=self.gstep)

    def create_summary(self):
        with tf.name_scope("summary"):
            tf.summary.scalar("contents_loss", self.content_loss)
            tf.summary.scalar("style_loss", self.style_loss)
            tf.summary.scalar("total_loss", self.total_loss)
            self.summary_op = tf.summary.merge_all()

    def build(self):
        self.create_input()
        self.load_vgg()
        self.losses()
        self.optimize()
        self.create_summary()

    def train(self, epoches=300):
        skip_step = 1
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            writer = tf.summary.FileWriter("graphs/style_transfer", sess.graph)
            
            sess.run(self.input_img.assign(self.initial_img))

            saver = tf.train.Saver()
            ckpt = tf.train.get_checkpoint_state('/content/gdrive/My Drive/MSc ML/0091/model.ckpt')
            if ckpt and ckpt.model_checkpoint_path:
                print("You have pre-trained model, if you do not want to use this, please delete the existing one.")
                saver.restore(sess, ckpt.model_checkpoint_path)

            initial_step = self.gstep.eval()

            for epoch in range(initial_step, epoches):
                # 前面几轮每隔10个epoch生成一张图片
                if epoch >= 5 and epoch < 20:
                    skip_step = 10
                # 后面每隔20个epoch生成一张图片
                elif epoch >= 20:
                    skip_step = 20
                
                sess.run(self.optimizer)
                
            gen_image, total_loss, summary = sess.run([self.input_img,
                                                       self.total_loss,
                                                       self.summary_op])
            # 对生成的图片逆向mean-center，即在每个channel上加上mean
            gen_image = gen_image + self.vgg.mean_pixels 
            writer.add_summary(summary, global_step=epoch)

            filename = "/content/gdrive/My Drive/MSc ML/0091/64x64/output10/" + str(self.ptr) + ".png"
            imageio.imwrite(filename, gen_image.reshape(gen_image.shape[1], gen_image.shape[2], gen_image.shape[3]))

            print("Step {}\n   Sum: {:5.1f}".format(epoch + 1, np.sum(gen_image)))
            print("   Loss: {:5.1f}".format(total_loss))
                        

In [0]:
mean = np.mean(A_input,0)

In [0]:
# B to A
for ptr in range(12,22):
    tf.reset_default_graph()

    input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_A")
    input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_B")

    conv1_1_B, conv2_1_B, conv3_1_B, conv4_1_B, conv4_2_B, conv5_1_B = feature_translator(input_A, name="translatorAtoB")
    conv1_1B, conv2_1B, conv3_1B, conv4_1B, conv4_2B, conv5_1B = feature_extractor(input_B, name="extractor")

    conv1_1_A, conv2_1_A, conv3_1_A, conv4_1_A, conv4_2_A, conv5_1_A = feature_translator(input_B, name="translatorBtoA")
    conv1_1A, conv2_1A, conv3_1A, conv4_1A, conv4_2A, conv5_1A = feature_extractor(input_A, name="extractor")


    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/content/gdrive/My Drive/MSc ML/0091/64x64/checkpoint/dense-1000/featuretrans-1000')
        conv1_1_A_temp, conv2_1_A_temp, conv3_1_A_temp, conv4_1_A_temp, conv4_2_A_temp, conv5_1_A_temp = sess.run([conv1_1_A, conv2_1_A, conv3_1_A, conv4_1_A, conv4_2_A, conv5_1_A], 
                                                                                                                  feed_dict={input_B:B_input[ptr]})


    # style transfer
    style_transfer = LatentTransfer(np.random.normal(127.5,10.0,(1,64,64,3)), conv1_1_A_temp, conv2_1_A_temp, conv3_1_A_temp, conv4_1_A_temp, conv5_1_A_temp, conv4_2_A_temp, img_width, img_height, ptr)
    style_transfer.build()
    style_transfer.train(100)

W0905 20:00:11.516843 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [104.69343392992019, 382.51709716796876]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2735359.4
   Loss: 222030656.0


W0905 20:00:28.901626 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [126.86334120178222, 370.55771606445313]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2765734.0
   Loss: 179247616.0


W0905 20:00:45.305198 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [133.97930990600585, 383.47550268554687]. Convert image to uint8 prior to saving to suppress this warning.


...

Step 100
   Sum: 2794467.8
   Loss: 211573712.0


W0905 20:02:27.157706 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [102.76862686157227, 371.09790502929684]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2958105.1
   Loss: 224787424.0


W0905 20:02:44.350972 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [121.78834789276124, 381.92566650390626]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2789086.0
   Loss: 238896640.0


In [0]:
for ptr in range(12, 22):
    tf.reset_default_graph()

    input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_A")
    input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_B")

    conv1_1_B, conv2_1_B, conv3_1_B, conv4_1_B, conv4_2_B, conv5_1_B = feature_translator(input_A, name="translatorAtoB")
    conv1_1B, conv2_1B, conv3_1B, conv4_1B, conv4_2B, conv5_1B = feature_extractor(input_B, name="extractor")

    conv1_1_A, conv2_1_A, conv3_1_A, conv4_1_A, conv4_2_A, conv5_1_A = feature_translator(input_B, name="translatorBtoA")
    conv1_1A, conv2_1A, conv3_1A, conv4_1A, conv4_2A, conv5_1A = feature_extractor(input_A, name="extractor")


    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/content/gdrive/My Drive/MSc ML/0091/64x64/checkpoint/dense-1000/featuretrans-1000')
        conv1_1_A_temp, conv2_1_A_temp, conv3_1_A_temp, conv4_1_A_temp, conv4_2_A_temp, conv5_1_A_temp = sess.run([conv1_1_B, conv2_1_B, conv3_1_B, conv4_1_B, conv4_2_B, conv5_1_B], 
                                                                                                                  feed_dict={input_A:A_input[ptr]})


    # style transfer
    style_transfer = LatentTransfer(np.random.normal(127.5,10.0,(1,64,64,3)), conv1_1_A_temp, conv2_1_A_temp, conv3_1_A_temp, conv4_1_A_temp, conv5_1_A_temp, conv4_2_A_temp, img_width, img_height, ptr)
    style_transfer.build()
    style_transfer.train(100)

W0905 20:07:24.640275 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [78.85172544860839, 409.48209448242187]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2814768.5
   Loss: 423641056.0


W0905 20:07:41.857168 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [89.26175962829589, 397.9545056152344]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2859920.4
   Loss: 375310976.0


W0905 20:07:59.002156 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [112.26409040832519, 412.2032849121094]. Convert image to uint8 prior to saving to suppress this warning.


...

Step 100
   Sum: 2887669.1
   Loss: 412234048.0


W0905 20:09:39.957492 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [103.4981403543949, 385.5727917480469]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 3029126.8
   Loss: 425245408.0


W0905 20:09:57.164739 140237731604352 util.py:61] Lossy conversion from float64 to uint8. Range [95.15247695922852, 398.9484020996094]. Convert image to uint8 prior to saving to suppress this warning.


Step 100
   Sum: 2835536.4
   Loss: 474607840.0
